In [1]:
import coremltools as ct

Torch version 2.2.2 has not been tested with coremltools. You may run into unexpected errors. Torch 2.1.0 is the most recent version that has been tested.


### Pytorch

In [2]:
import torch
import torchvision

In [3]:
torch_model = torchvision.models.mobilenet_v2()

### Tracing

In [4]:
torch_model.eval() # Inference mode
example_input = torch.rand(1, 3, 256, 256)
traced_model = torch.jit.trace(torch_model, example_input)

In [5]:
mlmodel = ct.convert(
    traced_model,
    source="pytorch", # Optional (for specific info)
    inputs=[ct.TensorType(shape=example_input.shape)], # PyTorch requires inputs.
    minimum_deployment_target=ct.target.iOS15 # iOS15, macOS12, watchOS8, tvOS15 -> MLProgram else MLModel
)

Running MIL default pipeline:   0%|                                                                                                             | 0/71 [00:00<?, ? passes/s]/Users/junnos/Diquest/CIFAR_Demo/converter/.venv/lib/python3.11/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:239: UserWarning: Input, 'x.1', of the source model, has been renamed to 'x_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL backend_mlprogram pipeline: 100%|████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 1769.38 passes/s]


In [6]:
print(mlmodel)

input {
  name: "x_1"
  type {
    multiArrayType {
      shape: 1
      shape: 3
      shape: 256
      shape: 256
      dataType: FLOAT32
    }
  }
}
output {
  name: "linear_0"
  type {
    multiArrayType {
      shape: 1
      shape: 1000
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==2.2.2"
  }
  userDefined {
    key: "com.github.apple.coremltools.source_dialect"
    value: "TorchScript"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "7.1"
  }
}



In [12]:
print(mlmodel.get_spec())

specificationVersion: 6
description {
  input {
    name: "x_1"
    type {
      multiArrayType {
        shape: 1
        shape: 3
        shape: 256
        shape: 256
        dataType: FLOAT32
      }
    }
  }
  output {
    name: "linear_0"
    type {
      multiArrayType {
        shape: 1
        shape: 1000
        dataType: FLOAT32
      }
    }
  }
  metadata {
    userDefined {
      key: "com.github.apple.coremltools.source"
      value: "torch==2.2.2"
    }
    userDefined {
      key: "com.github.apple.coremltools.source_dialect"
      value: "TorchScript"
    }
    userDefined {
      key: "com.github.apple.coremltools.version"
      value: "7.1"
    }
  }
}
mlProgram {
  version: 1
  functions {
    key: "main"
    value {
      inputs {
        name: "x_1"
        type {
          tensorType {
            dataType: FLOAT32
            rank: 4
            dimensions {
              constant {
                size: 1
              }
            }
            dimensions {

In [14]:
spec = mlmodel.get_spec()
ct.utils.rename_feature(spec, "x_1", "input")
ct.utils.rename_feature(spec, "linear_0", "output")
mlmodel = ct.models.MLModel(spec, weights_dir=mlmodel.weights_dir) # If model is a MLProgram, directory of weight is also needed.

In [16]:
mlmodel.short_description = "Image Classifier"
mlmodel.license = "Apache 2.0"
mlmodel.author = "Junnos"

In [20]:
mlmodel.save("./model/mobilenet.mlpackage")